#Data Preparation using Std Scaler for Numerical data and get dummies for categorical data <B>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

import joblib

In [2]:
data=pd.read_csv("data.csv")

------------------------------Data Preparation-------------------------------
----------------------------------------------------------------------------------------


In [3]:
data

,Item_ID,Item_W,Item_Type,Item_MRP,Outlet_ID,Outlet_Size,Outlet_Location_Type,Sales,data,Item_ID_group,Outlet_Age
0,FDU32,21.027499,Baking Goods,197.352319,OUT046,Small,Tier 2,2689.457781,train,FDU,17
1,NCT54,21.102371,Meat,148.250214,OUT035,Small,Tier 1,3437.350375,train,NCT,34
2,FDW08,20.882263,Hard Drinks,205.465010,OUT035,Small,Tier 3,3129.967268,train,FDW,22
3,FDJ22,21.050435,Starchy Foods,253.417583,OUT046,Small,Tier 1,1306.514376,train,FDJ,25
4,FDF47,21.247876,Baking Goods,240.871039,OUT035,Small,Tier 3,1739.769829,train,FDF,33
...,...,...,...,...,...,...,...,...,...,...,...
125515,FDY27,6.380000,Dairy,178.334400,OUT018,Medium,Tier 3,NaN,test,FDY,12
125516,FDK38,6.650000,Canned,150.073400,OUT018,Medium,Tier 3,NaN,test,FDK,12
125517,DRF01,5.655000,Soft Drinks,147.510200,OUT046,Small,Tier 1,NaN,test,DRF,24
125518,FDR58,6.675000,Snack Foods,92.946200,OUT018,Medium,Tier 3,NaN,test,FDR,12


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125520 entries, 0 to 125519
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Item_ID               125520 non-null  object 
 1   Item_W                125520 non-null  float64
 2   Item_Type             125520 non-null  object 
 3   Item_MRP              125520 non-null  float64
 4   Outlet_ID             125520 non-null  object 
 5   Outlet_Size           125520 non-null  object 
 6   Outlet_Location_Type  125520 non-null  object 
 7   Sales                 87864 non-null   float64
 8   data                  125520 non-null  object 
 9   Item_ID_group         125520 non-null  object 
 10  Outlet_Age            125520 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 10.5+ MB


In [5]:
numeric_data_columns = data.select_dtypes(include = np.number).drop(columns = "Sales").columns

In [6]:
category_data_columns = data.select_dtypes(exclude = np.number).drop(columns=["Item_ID","data"]).columns

In [7]:
numeric_data_columns ,

(Index(['Item_W', 'Item_MRP', 'Outlet_Age'], dtype='object'),)

In [8]:
category_data_columns

Index(['Item_Type', 'Outlet_ID', 'Outlet_Size', 'Outlet_Location_Type',
       'Item_ID_group'],
      dtype='object')

In [9]:
preprocess_category_data_columns = pd.get_dummies(data[category_data_columns])

In [10]:
scaler = StandardScaler()

In [11]:
preprocess_numeric_data_columns = pd.DataFrame(scaler.fit_transform(data[numeric_data_columns]), columns=numeric_data_columns)

In [12]:
preprocess_numeric_data_columns.isna().sum()

Item_W        0
Item_MRP      0
Outlet_Age    0
dtype: int64

In [13]:
preprocess_category_data_columns.isna().sum()

Item_Type_Baking Goods    0
Item_Type_Breads          0
Item_Type_Breakfast       0
Item_Type_Canned          0
Item_Type_Dairy           0
                         ..
Item_ID_group_NCV         0
Item_ID_group_NCW         0
Item_ID_group_NCX         0
Item_ID_group_NCY         0
Item_ID_group_NCZ         0
Length: 97, dtype: int64

In [14]:
final_preprocessed = pd.concat([preprocess_category_data_columns, preprocess_numeric_data_columns,data[["Sales","Item_ID","data"]]], axis = 1)

In [15]:
final_preprocessed

,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,...,Item_ID_group_NCW,Item_ID_group_NCX,Item_ID_group_NCY,Item_ID_group_NCZ,Item_W,Item_MRP,Outlet_Age,Sales,Item_ID,data
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.410465,0.792848,-0.608131,2689.457781,FDU32,train
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.427374,-0.022995,1.925332,3437.350375,NCT54,train
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1.377664,0.927642,0.137005,3129.967268,FDW08,train
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.415644,1.724386,0.584087,1306.514376,FDJ22,train
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1.460236,1.515922,1.776305,1739.769829,FDF47,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125515,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,-1.897623,0.476861,-1.353268,NaN,FDY27,test
125516,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,-1.836644,0.007297,-1.353268,NaN,FDK38,test
125517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-2.061362,-0.035291,0.435059,NaN,DRF01,test
125518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,-1.830998,-0.941885,-1.353268,NaN,FDR58,test


In [16]:
train_preprocessed = final_preprocessed[final_preprocessed["data"]=="train"]

In [17]:
test_preprocessed = final_preprocessed[final_preprocessed["data"]=="test"]

In [18]:
test_preprocessed = test_preprocessed.drop(columns = ["data","Sales"])

In [19]:
train_preprocessed = train_preprocessed.drop(columns = ["data"])

In [20]:
test_preprocessed

,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,...,Item_ID_group_NCU,Item_ID_group_NCV,Item_ID_group_NCW,Item_ID_group_NCX,Item_ID_group_NCY,Item_ID_group_NCZ,Item_W,Item_MRP,Outlet_Age,Item_ID
87864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.044663,-0.503686,0.137005,DRM23
87865,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,-0.981945,1.893577,1.925332,FDG47
87866,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,-0.638847,1.605909,0.435059,FDN21
87867,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.037554,-0.223204,-0.608131,FDZ23
87868,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,-0.836135,0.390649,-1.353268,FDC16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125515,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,-1.897623,0.476861,-1.353268,FDY27
125516,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.836644,0.007297,-1.353268,FDK38
125517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-2.061362,-0.035291,0.435059,DRF01
125518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-1.830998,-0.941885,-1.353268,FDR58


In [21]:
train_preprocessed

,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,...,Item_ID_group_NCV,Item_ID_group_NCW,Item_ID_group_NCX,Item_ID_group_NCY,Item_ID_group_NCZ,Item_W,Item_MRP,Outlet_Age,Sales,Item_ID
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.410465,0.792848,-0.608131,2689.457781,FDU32
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.427374,-0.022995,1.925332,3437.350375,NCT54
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1.377664,0.927642,0.137005,3129.967268,FDW08
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.415644,1.724386,0.584087,1306.514376,FDJ22
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.460236,1.515922,1.776305,1739.769829,FDF47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87859,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,-1.081320,-0.037290,1.925332,4975.646052,FDG24
87860,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,-0.795661,-0.360028,1.925332,2252.492755,FDA32
87861,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,-0.767077,-0.508514,0.435059,2716.058050,FDM04
87862,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0.730786,1.225908,1.776305,991.857233,NCS18


In [22]:
train_preprocessed.to_csv("train_preprocessed1.csv", index = False)
test_preprocessed.to_csv("test_preprocessed1.csv", index = False)